In [1]:
# importing libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
sns.set_style("darkgrid")
plt.style.use("fivethirtyeight")
import os
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,LabelEncoder,PowerTransformer
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import f_regression,RFECV
from sklearn.linear_model import ElasticNet,Ridge,Lasso
from scipy.special import boxcox1p
from sklearn.linear_model import LinearRegression
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
from sklearn.compose import ColumnTransformer
import lightgbm as lgm
! pip install category_encoders
from category_encoders import TargetEncoder
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.ensemble import StackingRegressor
import warnings
warnings.simplefilter('ignore')
from category_encoders import TargetEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder
import scipy as sc
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import RFE
import lightgbm as lgm
import math

     |████████████████████████████████| 80 kB 4.9 MB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [139]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [141]:
mean_gridlivarea=train.loc[train['GrLivArea']>4500,"SalePrice"].mean()
mean_masvnrarea=train.loc[train['MasVnrArea']>1200,"SalePrice"].mean()
mean_bsmtfin2=train.loc[train['BsmtFinSF2']>1400,"SalePrice"].mean()
mean_woodeck=train.loc[train['WoodDeckSF']>800,"SalePrice"].mean()
mean_openporch=train.loc[train['OpenPorchSF']>500,"SalePrice"].mean()
mean_enclosedporch=train.loc[train['EnclosedPorch']>500,"SalePrice"].mean()
outlier_indices=[934,1298,249,313,335,706,205,55]

In [142]:
def GridSearch(model,param_grid,X,y):
    grid=GridSearchCV(model,param_grid,cv=10,scoring='neg_mean_squared_error')
    grid.fit(X,y)
    print(np.sqrt(-grid.best_score_))
    print(grid.best_params_)
def cross_validate(model,X,y):
    scores=cross_val_score(model,X,y,scoring='neg_mean_squared_error')
    print(np.sqrt(-scores.mean()))
def create_submission(predictions,name):
    submission=pd.DataFrame()
    submission['Id']=test_idx
    submission['SalePrice']=predictions
    submission.to_csv(name+".csv",index=False)
def Numerical_Summary(dataframe):
    summary=pd.DataFrame()
    for col in dataframe.columns:
        if dataframe[col].dtype==object:
            continue
        row={"Column":col,"Missing_Values":dataframe[col].isnull().sum(),"Variance":dataframe[col].var(),"Skewness":dataframe[col].skew()}
        summary=summary.append(pd.Series(row),ignore_index=True)
    
    return summary
def categorical_summary(dataframe):
    print("Categorical Features Summary:-")
    print("="*50)
    for col in dataframe.columns:
        if dataframe[col].dtype!=object:
            continue
        print("Column =",col)
        print("Value Counts:-",dataframe[col].value_counts())
        print("Missing Values=",dataframe[col].isnull().sum())
        print("="*50)
numerical_columns=[col for col in train.columns if train[col].dtype!=object and col!='SalePrice']
categorical_columns=[col for col in train.columns if train[col].dtype==object]
#outlier_indices=[934,249,313,335,706,1298]
skewed_features=[col for col in train.columns if col not in categorical_columns and abs(train[col].skew())>0.5 and col!='SalePrice'  ]
def scatterplots():
    fig,ax=plt.subplots(9,2,figsize=(30,30))
    sns.scatterplot(train['LotFrontage'],train['SalePrice'],ax=ax[0,0])
    sns.scatterplot(train['LotArea'],train['SalePrice'],ax=ax[0,1])
    sns.scatterplot(train['MasVnrArea'],train['SalePrice'],ax=ax[1,0])
    sns.scatterplot(train['GrLivArea'],train['SalePrice'],ax=ax[1,1])
    sns.scatterplot(train['BsmtFinSF1'],train['SalePrice'],ax=ax[2,0])
    sns.scatterplot(train['BsmtFinSF2'],train['SalePrice'],ax=ax[2,1])
    sns.scatterplot(train['1stFlrSF'],train['SalePrice'],ax=ax[3,0])
    sns.scatterplot(train['2ndFlrSF'],train['SalePrice'],ax=ax[3,1])
    sns.scatterplot(train['GarageArea'],train['SalePrice'],ax=ax[4,0])
    sns.scatterplot(train['TotalBsmtSF'],train['SalePrice'],ax=ax[4,1])
    sns.scatterplot(train['YearBuilt'],train['SalePrice'],ax=ax[5,0])
    sns.scatterplot(train['BsmtUnfSF'],train['SalePrice'],ax=ax[5,1])
    sns.scatterplot(train['LowQualFinSF'],train['SalePrice'],ax=ax[6,0])
    sns.scatterplot(train['GarageArea'],train['SalePrice'],ax=ax[6,1])
    sns.scatterplot(train['WoodDeckSF'],train['SalePrice'],ax=ax[7,0])
    sns.scatterplot(train['OpenPorchSF'],train['SalePrice'],ax=ax[7,1])
    sns.scatterplot(train['EnclosedPorch'],train['SalePrice'],ax=ax[8,0])
    sns.scatterplot(train['3SsnPorch'],train['SalePrice'],ax=ax[8,1])
    
    
def Impute_Missing(dataframe):
    df=dataframe.copy()
    for col in ["Alley",'MasVnrType','PoolQC','Fence', 'MiscFeature',"FireplaceQu",'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','GarageType','GarageQual',
       'GarageCond','GarageFinish','BsmtQual']:
        df[col].fillna("Missing",inplace=True)
    for col in ['MSZoning','Exterior1st','Exterior2nd','Electrical','KitchenQual','Functional',"SaleType","Utilities"]:
        df[col].fillna(df[col].mode()[0],inplace=True)
    for col in ['MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','GarageYrBlt','BsmtFullBath','BsmtHalfBath','GarageCars','GarageArea']:
        df[col].fillna(0,inplace=True)
    df['LotFrontage']=dataframe.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    return df
def Heatmap(dataframe):
    plt.figure(figsize=(20,20))
    sns.heatmap(dataframe,annot=True)
def Fix_Skew(dataframe):
    df=dataframe.copy()
    for col in skewed_features:
        df[col]=boxcox1p(df[col],0.15)
    return df
def Feature_Engineering(dataframe):
    all_data=dataframe.copy()
    all_data['Total_Living_Sq_Area']=all_data['BsmtFinSF1']+all_data['BsmtFinSF2']+all_data['1stFlrSF']+all_data['2ndFlrSF']
    all_data['Total_House_Area']=all_data['TotalBsmtSF']+all_data['1stFlrSF']+all_data['2ndFlrSF']
    all_data['Total_Bathrooms']=all_data['BsmtFullBath']+0.5*all_data['BsmtHalfBath']+0.5*all_data['HalfBath']+all_data['FullBath']
    all_data['Total_Porch']=all_data['WoodDeckSF']+all_data['OpenPorchSF']+all_data['EnclosedPorch']+all_data['3SsnPorch']+all_data['ScreenPorch']
    all_data['Has_Pool']=all_data['PoolQC'].apply(lambda x: 1 if x is np.nan else 0)
    all_data['Has_Alley']=all_data['Alley'].apply(lambda x: 1 if x is  np.nan else 0)
    all_data['Has_Fireplace']=all_data['FireplaceQu'].apply(lambda x: 1 if x is  np.nan else 0)
    all_data['Age_House']=all_data['YrSold']-all_data['YearBuilt']
    all_data['Years_After_Remodal']=all_data['YrSold']-all_data['YearRemodAdd']
    return all_data
test_idx=test['Id'].copy()
def remove_id(dataframe):
    dataframe.drop("Id",axis=1,inplace=True)
def create_submission(predictions,name):
    df=pd.DataFrame()
    df['Id']=test_idx
    df['SalePrice']=predictions
    df.to_csv(name+".csv",index=False)
def create_target(dataframe):
    dataframe['SalePrice']=np.log1p(dataframe['SalePrice'])
    y=dataframe['SalePrice'].copy()
    dataframe.drop("SalePrice",axis=1,inplace=True)
    return y
def feature_selection(dataframe,ncols):
    targetencoder=TargetEncoder(cols=categorical_columns)
    train_data=dataframe[numerical_columns].join(targetencoder.fit_transform(dataframe[categorical_columns],dataframe['SalePrice']))
    dtree=ExtraTreesRegressor()
    dtree.fit(train_data,dataframe["SalePrice"])
    feature_importances=pd.DataFrame(sorted(zip(dtree.feature_importances_,train_data),reverse=True),columns=['Importance',"Feature"])
    cols=list(feature_importances.loc[:ncols,"Feature"])
    return cols
def feature_selection_RFE(dataframe,cols):
    rfe=RFE(ExtraTreesRegressor(),n_features_to_select=cols)
    targetencoder=TargetEncoder(cols=categorical_columns)
    train_data=dataframe[numerical_columns].join(targetencoder.fit_transform(dataframe[categorical_columns],dataframe['SalePrice']))
    rfe.fit(train_data,dataframe['SalePrice'])
    features_selected=pd.DataFrame(zip(rfe.support_,train_data.columns),columns=["Is_Selected","Column"])
    feature_selected=features_selected[features_selected['Is_Selected']==True]['Column']
    return list(feature_selected)


In [143]:
# Removing Outlier Rows
train=train.drop(index=outlier_indices).reset_index().drop("index",axis=1)
#mean=train
train=train[train['GrLivArea']<4500].reset_index(drop=True)
print("REMOVING OUTLIER ROWS DONE")
print("="*50)

# Missing Values Imputation
train=Impute_Missing(train)
test=Impute_Missing(test)
print("MISSING VALUES IMPUTATION  DONE")
print("="*50)

# Removing Skew 
train=Fix_Skew(train)
test=Fix_Skew(test)
print("FIXING SKEW FEATURES DONE")
print("="*50)

# Feature Engineering
train=Feature_Engineering(train)
test=Feature_Engineering(test)
print("FEATURE ENGINEERING DONE")
print("="*50)

# Feature_selection
cols=feature_selection(train,59)
train=train[cols+["SalePrice"]].copy()
test=test[cols].copy()
print("FEATURE SELECTION DONE")
print("="*50)

# Creating Target Variable
y=create_target(train)
new_cat_cols=[col for col in cols if train[col].dtype==object]
new_numerical_cols=[col for col in cols if train[col].dtype!=object]

# getting dumies
final_pipeline=ColumnTransformer(transformers=[("Numerical",RobustScaler(),new_numerical_cols),("Categorical",OneHotEncoder(handle_unknown="ignore"),new_cat_cols)])

print("DATA IS READY FOR MODELLING")
print("="*50)
print("Training Data Shape after preprocessing for modelling",train.shape)
print("Testing Data Shape after preprocessing for modelling",test.shape)




REMOVING OUTLIER ROWS DONE
MISSING VALUES IMPUTATION  DONE
FIXING SKEW FEATURES DONE
FEATURE ENGINEERING DONE
FEATURE SELECTION DONE
DATA IS READY FOR MODELLING
Training Data Shape after preprocessing for modelling (1451, 60)
Testing Data Shape after preprocessing for modelling (1459, 60)


In [144]:
X=final_pipeline.fit_transform(train)
print(X.shape)

(1451, 232)


**Linear Models**

In [145]:
GridSearch(Lasso(),{"alpha":[0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009]},X,y)

0.11124056462731058
{'alpha': 0.0003}


In [146]:
GridSearch(Ridge(),{"alpha":[30,31,32,33,34,35,16,20,15,12]},X,y)

0.11386347660000201
{'alpha': 15}


In [103]:

GridSearch(ElasticNet(alpha=0.001),{"l1_ratio":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},X,y)

0.11085807489455071
{'l1_ratio': 0.3}


In [147]:
lasso_predictions=np.zeros((len(test),1))
y_train_pred_lasso=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    lasso=Lasso(alpha=0.0003)
    lasso.fit(x_train,y[train_idx])
    p=lasso.predict(x_val)
    
    y_train_pred_lasso[val_idx]+=np.reshape(p,(len(p),1))
    p=lasso.predict(x_test)
    lasso_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for lasso =",np.sqrt(mean_squared_error(y,y_train_pred_lasso
                                                                     )))

Cross Validation RMSE for lasso = 0.11154276631612817


In [148]:
ridge_predictions=np.zeros((len(test),1))
y_train_pred_ridge=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
  x_train=final_pipeline.fit_transform(train.loc[train_idx])
  x_valid=final_pipeline.transform(train.loc[val_idx])
  x_test=final_pipeline.transform(test)
  ridge=Ridge(alpha=15)
  ridge.fit(x_train,y[train_idx])
  p=ridge.predict(x_valid)
  y_train_pred_ridge[val_idx]+=np.reshape(p,(len(p),1))
  p=ridge.predict(x_test)
  ridge_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for Ridge =",np.sqrt(mean_squared_error(y,y_train_pred_ridge)))

Cross Validation RMSE for Ridge = 0.11438499948722117


In [118]:
en_predictions=np.zeros((len(test),1))
y_train_pred_en=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
  x_train=final_pipeline.fit_transform(train.loc[train_idx])
  x_valid=final_pipeline.transform(train.loc[val_idx])
  x_test=final_pipeline.transform(test)
  en=ElasticNet(alpha=0.001,l1_ratio=0.3)
  en.fit(x_train,y[train_idx])
  p=en.predict(x_valid)
  y_train_pred_en[val_idx]+=np.reshape(p,(len(p),1))
  p=en.predict(x_test)
  en_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for ElasticNet =",np.sqrt(mean_squared_error(y,y_train_pred_en)))

Cross Validation RMSE for ElasticNet = 0.11123945637487202


**Boosting Models**

In [43]:
GridSearch(GradientBoostingRegressor(max_depth=3,max_features="sqrt",loss='huber'),{'n_estimators':[1000,3000,2000],"learning_rate":[0.01,0.05]},X,y)

0.11185687687797413
{'learning_rate': 0.05, 'n_estimators': 2000}


In [44]:
GridSearch(XGBRegressor(objective='reg:squarederror',n_estimators=1000,learning_rate=0.05,max_depth=3,gamma=0.01,colsample_bytree=0.8),{'subsample':[0.8,0.9]},X,y)

0.11533712410051827
{'subsample': 0.9}


In [ ]:

GridSearch(lgm.LGBMRegressor(learning_rate=0.05,n_estimators=750,num_leaves=5,max_depth=3,
                            objective="regression",subsample=0.5,colsample_bytree=0.5),{"max_depth":[3,5,10,8]},X,y)

0.11772169315744874
{'max_depth': 3}


In [119]:
gb_predictions=np.zeros((len(test),1))
y_train_pred_gb=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
  x_train=final_pipeline.fit_transform(train.loc[train_idx])
  x_valid=final_pipeline.transform(train.loc[val_idx])
  x_test=final_pipeline.transform(test)
  gb=GradientBoostingRegressor(n_estimators=3000,learning_rate=0.01,loss='huber',max_depth=3,max_features='sqrt')
  gb.fit(x_train,y[train_idx])
  p=gb.predict(x_valid)
  y_train_pred_gb[val_idx]+=np.reshape(p,(len(p),1)) 
  p=gb.predict(x_test)
  gb_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for GradientBoosting =",np.sqrt(mean_squared_error(y,y_train_pred_gb)))

Cross Validation RMSE for GradientBoosting = 0.11367758188435981


In [120]:
xgboost_predictions=np.zeros((len(test),1))
y_train_pred_xgboost=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    xgboost = XGBRegressor(objective='reg:squarederror',n_estimators=1000,learning_rate=0.05,max_depth=3,gamma=0.01,colsample_bytree=0.8,subsample=0.8)
    xgboost.fit(x_train,y[train_idx])
    p=xgboost.predict(x_val)
    
    y_train_pred_xgboost[val_idx]+=np.reshape(p,(len(p),1))
    p=xgboost.predict(x_test)
    xgboost_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for Xgboost =",np.sqrt(mean_squared_error(y,y_train_pred_xgboost)))

Cross Validation RMSE for Xgboost = 0.11782287169291854


In [149]:
light_gbm_predictions=np.zeros((len(test),1))
y_train_pred_gbm=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    model_lgb = lgm.LGBMRegressor(learning_rate=0.05,n_estimators=750,num_leaves=5,max_depth=3,
                            objective="regression",subsample=0.5,colsample_bytree=0.5)
    model_lgb.fit(x_train,y[train_idx])
    p=model_lgb.predict(x_val)
    
    y_train_pred_gbm[val_idx]+=np.reshape(p,(len(p),1))
    p=model_lgb.predict(x_test)
    light_gbm_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for LightGbm =",np.sqrt(mean_squared_error(y,y_train_pred_gbm)))

Cross Validation RMSE for LightGbm = 0.11961958993912819


**Support Vector Machines**

In [ ]:
GridSearch(SVR(gamma=0.0003,epsilon=0.001),{"C":[5,10,15,12,18]},X,y)

0.11392404796806038
{'C': 18}


In [48]:
svr_predictions=np.zeros((len(test),1))
y_train_pred_svr=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    svr = SVR(C=15,epsilon=0.001,gamma=0.0003)
    svr.fit(x_train,y[train_idx])
    p=svr.predict(x_val)
    
    y_train_pred_svr[val_idx]+=np.reshape(p,(len(p),1))
    p=svr.predict(x_test)
    svr_predictions+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for Support Vector Macines =",np.sqrt(mean_squared_error(y,y_train_pred_svr)))

Cross Validation RMSE for Support Vector Macines = 0.11579710917188761


**StackedCVRegressor**

In [150]:
svr = SVR(C=18,epsilon=0.001,gamma=0.0003)
model_lgb = lgm.LGBMRegressor(learning_rate=0.05,n_estimators=750,num_leaves=5,max_depth=3,
                            objective="regression",subsample=0.5,colsample_bytree=0.5)
ridge=Ridge(alpha=15)
lasso=Lasso(alpha=0.0004)
en=ElasticNet(alpha=0.001,l1_ratio=0.3)
sr_predictions1=np.zeros((len(test),1))
xgboost = XGBRegressor(objective='reg:squarederror',n_estimators=1000,learning_rate=0.01,max_depth=3,gamma=0.01,colsample_bytree=0.8,subsample=0.9)
y_train_pred_sr1=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    model=StackingRegressor(estimators=[("Support Vector Machines",svr),("Ridge",ridge),("LightGBM",model_lgb)],final_estimator=LinearRegression())

    model.fit(x_train,y[train_idx])
    p=model.predict(x_val)
    
    y_train_pred_sr1[val_idx]+=np.reshape(p,(len(p),1))
    p=model.predict(x_test)
    sr_predictions1+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for Stacking Regressor =",np.sqrt(mean_squared_error(y,y_train_pred_sr1)))

Cross Validation RMSE for Stacking Regressor = 0.11103321301954662


In [152]:
svr = SVR(C=18,epsilon=0.001,gamma=0.0003)
model_lgb = lgm.LGBMRegressor(learning_rate=0.05,n_estimators=750,num_leaves=5,max_depth=3,
                            objective="regression",subsample=0.5,colsample_bytree=0.5)
ridge=Ridge(alpha=16)
lasso=Lasso(alpha=0.0004)
en=ElasticNet(alpha=0.001,l1_ratio=0.3)
sr_predictions2=np.zeros((len(test),1))
xgboost = XGBRegressor(objective='reg:squarederror',n_estimators=1000,learning_rate=0.01,max_depth=3,gamma=0.01,colsample_bytree=0.8,subsample=0.9)
y_train_pred_sr2=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    model=StackingRegressor(estimators=[("Support Vector Machines",svr),("Lasso",lasso),("LightGBM",model_lgb)],final_estimator=LinearRegression())

    model.fit(x_train,y[train_idx])
    p=model.predict(x_val)
    
    y_train_pred_sr2[val_idx]+=np.reshape(p,(len(p),1))
    p=model.predict(x_test)
    sr_predictions2+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for Stacking Regressor =",np.sqrt(mean_squared_error(y,y_train_pred_sr2)))

Cross Validation RMSE for Stacking Regressor = 0.10964565767368856


In [156]:
svr = SVR(C=18,epsilon=0.001,gamma=0.0003)
model_lgb = lgm.LGBMRegressor(learning_rate=0.05,n_estimators=750,num_leaves=5,max_depth=3,
                            objective="regression",subsample=0.5,colsample_bytree=0.5)
ridge=Ridge(alpha=16)
lasso=Lasso(alpha=0.0004)
en=ElasticNet(alpha=0.001,l1_ratio=0.5)
sr_predictions3=np.zeros((len(test),1))
xgboost = XGBRegressor(objective='reg:squarederror',n_estimators=1000,learning_rate=0.01,max_depth=3,gamma=0.01,colsample_bytree=0.8,subsample=0.9)
y_train_pred_sr3=np.zeros((len(train),1))
kfold=KFold(random_state=42)
for idx,(train_idx,val_idx) in enumerate(kfold.split(train.values,y)):
    x_train=final_pipeline.fit_transform(train.loc[train_idx])
    x_val=final_pipeline.transform(train.loc[val_idx])
    x_test=final_pipeline.transform(test)
    model=StackingRegressor(estimators=[("Support Vector Machines",svr),("ElasticNet",en),("LightGBM",model_lgb)],final_estimator=LinearRegression())

    model.fit(x_train,y[train_idx])
    p=model.predict(x_val)
    
    y_train_pred_sr3[val_idx]+=np.reshape(p,(len(p),1))
    p=model.predict(x_test)
    sr_predictions3+=np.reshape(p,(len(p),1))/5
print("Cross Validation RMSE for Stacking Regressor =",np.sqrt(mean_squared_error(y,y_train_pred_sr3)))

Cross Validation RMSE for Stacking Regressor = 0.10991730710799671


In [157]:
predictions=sr_predictions3
predictions=np.expm1(predictions)
predictions[1089]=mean_gridlivarea
#predictions[523]=mean_masvnrarea
#predictions[1130]=mean_masvnrarea
#predictions[1026]=mean_openporch
#predictions[1097]=mean_openporch
create_submission(predictions,"Lasso")